## Topic Modeling - Wissenschaft-Datensatz

In [ ]:
!pip install bertopic

In [40]:
from bertopic import BERTopic

import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Daten einlesen & Preprocessing

In [41]:
df = pd.read_excel('tweets_wissenschaft.xlsx')

In [42]:
df['tweet_text'] = df['tweet_text'].astype(str)
df['retweet_text_retweeted_tweet'] = df['retweet_text_retweeted_tweet'].astype(str)
df['zitat_text'] = df['zitat_text'].astype(str)

In [43]:
def classify_tweet(row):
    if row['tweet_text'] != '-998' and row['retweet_text_retweeted_tweet'] == '-998' and row['zitat_text'] == '-998':
        return 'tweet'
    elif row['retweet_text_retweeted_tweet'] != '-998':
        return 'retweet'
    elif row['tweet_text'] != '-998' and row['zitat_text'] != '-998':
        return 'zitat_tweet'
    return 'unclassified'

In [44]:
df['tweet_class'] = df.apply(classify_tweet, axis=1)

In [45]:
def gather_texts(row):
    texts = []
    for column in ['tweet_text', 'retweet_text_retweeted_tweet', 'zitat_text']:
      if pd.notna(row[column]) and str(row[column]) != '-998':
            texts.append(row[column])
    return ' '.join(texts)

In [46]:
df['all_tweets'] = df.apply(gather_texts, axis=1)

In [47]:
def preprocess_tweet(text):
    text = re.sub(r'@\w+', '', text)  # Entfernt Benutzernamen
    text = re.sub(r'http\S+', '', text)  # Entfernt URLs
    text = re.sub(r'www.\S+', '', text)  # Entfernt URLs
    text = re.sub(r'[0-9]+', '', text)  # Entfernt Zahlen
    text = re.sub(r'[^\w\s]', '', text)  # Entfernt Sonderzeichen
    text = re.sub(r'_', '', text)  # Entfernt Unterstriche
    text = re.sub(r'\s+', ' ', text).strip()  # Ersetzt mehrere Leerzeichen durch ein einzelnes und entfernt sie am Anfang/Ende
    text = re.sub(r'-998', '', text)  # entfernt fehlende Werte
    text = re.sub(r'nan', '', text)  # entfernt fehlende Werte

    return text

In [48]:
df['all_tweets'] = df['all_tweets'].apply(preprocess_tweet)

### Topic Modeling mit BERTopic

In [49]:
docs = list(df.all_tweets.values)

german_stopwords = set(stopwords.words('german'))
stopwords = list(german_stopwords)
vectorizer_model = CountVectorizer(stop_words=stopwords)

In [50]:
len(docs)

3749

#### Erstes Modell ohne Beschränkung der Themenanzahl



In [51]:
# BERTopic-Modell erstellen
model = BERTopic(language="German", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)

# Model anwenden und Topics extrahieren
topics, probs = model.fit_transform(docs)

2024-04-29 11:35:43,291 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/118 [00:00<?, ?it/s]

2024-04-29 11:35:54,121 - BERTopic - Embedding - Completed ✓
2024-04-29 11:35:54,127 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 11:36:26,285 - BERTopic - Dimensionality - Completed ✓
2024-04-29 11:36:26,287 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 11:36:28,090 - BERTopic - Cluster - Completed ✓
2024-04-29 11:36:28,097 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 11:36:28,549 - BERTopic - Representation - Completed ✓


In [52]:
# Ergebnisse
print(f"Anzahl der Topics: {model.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info = model.get_topic_info()
topic_info.head(20)

Anzahl der Topics: 101


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1121,-1_forschung_heute_gute_gibt,"[forschung, heute, gute, gibt, innovation, mac...",[NATURE beklagt den Rückgang innovativer Forsc...
1,0,222,0_forschung_ja_freiheit_immer,"[forschung, ja, freiheit, immer, gibt, einfach...",[Eigentlich geht das ganz einfach Es muß etwas...
2,1,129,1_deutschland_deutsche_deutschen_beschlossen,"[deutschland, deutsche, deutschen, beschlossen...",[Deutschland wird jetzt schneller und innovati...
3,2,110,2_mecfs_versorgung_longcovid_mecfsimbundestag,"[mecfs, versorgung, longcovid, mecfsimbundesta...",[Heute stille Visualisierung vor dem Bundestag...
4,3,63,3_desamfornet_medizin_klinik_data,"[desamfornet, medizin, klinik, data, mitmachen...",[Brücken zwischen Forschung und Klinik stärken...
5,4,51,4_digitalen_digitalisierung_transformation_cyb...,"[digitalen, digitalisierung, transformation, c...",[Die stärkt Forschung und Lehre zur digitalen ...
6,5,45,5_politiksprüche_zib_verbote_verzweifeln,"[politiksprüche, zib, verbote, verzweifeln, um...",[Immer wieder dieselben Politiksprüche beim Kl...
7,6,42,6_ausgezeichnet_gratulieren_award_leistungen,"[ausgezeichnet, gratulieren, award, leistungen...",[Für seine Leistungen im Bereich der Forschung...
8,7,41,7_archäologie_germanen_meldungen_kultur,"[archäologie, germanen, meldungen, kultur, nac...",[Waren die Germanen nur raubeinige Barbaren Di...
9,8,39,8_genderstudies_personen_reagierst_nervtötendster,"[genderstudies, personen, reagierst, nervtöten...",[Wenn du über Jahre ach was Jahrzehnte in nerv...


In [53]:
model.visualize_topics()

#### Reduzierung der Themenanzahl

In [54]:
model.reduce_topics(docs, nr_topics=30)

2024-04-29 11:36:30,627 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 11:36:30,965 - BERTopic - Topic reduction - Reduced number of topics from 102 to 30


In [55]:
# Ergebnisse
print(f"Anzahl der Topics: {model.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info_reduced = model.get_topic_info()
topic_info_reduced

Anzahl der Topics: 29


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1121,-1_forschung_heute_gibt_mehr,"[forschung, heute, gibt, mehr, gute, innovatio...",[Jeden Tag kommen so unglaublich viele spannen...
1,0,736,0_forschung_ichbinhanna_wissenschaft_lehre,"[forschung, ichbinhanna, wissenschaft, lehre, ...",[Immer wieder dieselben Politiksprüche beim Kl...
2,1,328,1_forschung_digitale_the_lehre,"[forschung, digitale, the, lehre, thema, chatg...",[Auch in dieser Woche bieten wir Ihnen wieder ...
3,2,294,2_mecfs_versorgung_mecfsimbundestag_forschung,"[mecfs, versorgung, mecfsimbundestag, forschun...",[MECFSimBundestag Heute nach Jahrzehnten ohne ...
4,3,164,3_berlin_freuen_bewerbungen_exzellente,"[berlin, freuen, bewerbungen, exzellente, heim...",[Wusstet ihr dass Berlin DER Heimatort von Wis...
5,4,141,4_heißt_longcovid_covid_viren,"[heißt, longcovid, covid, viren, prävention, e...",[Endemie heißt nicht dass ein Erreger harmlos ...
6,5,120,5_kernfusion_wasserstoff_alternative_energie,"[kernfusion, wasserstoff, alternative, energie...",[Und was ist die Alternative in der Akutheit d...
7,6,113,6_geht_klima_engagiert_ehren,"[geht, klima, engagiert, ehren, fleissig, klim...",[Manche Menschen verstehen einfach nicht was d...
8,7,106,7_frauen_genderstudies_mädchen_wissenschaftler...,"[frauen, genderstudies, mädchen, wissenschaftl...",[Wenn du über Jahre ach was Jahrzehnte in nerv...
9,8,84,8_käthe_ukraine_film_leichterstaatspreis,"[käthe, ukraine, film, leichterstaatspreis, jü...",[Es ist nur eine Nebenepisode aber symbolpolit...


In [56]:
for i in range(1, 11):
    representation = topic_info_reduced.loc[i, 'Representation']
    representative_docs = topic_info_reduced.loc[i, 'Representative_Docs']
    print(f"Thema {i}:")
    print(f"Representation: {representation}")
    print(f"Representative_Docs: {representative_docs}")
    print()

Thema 1:
Representation: ['forschung', 'ichbinhanna', 'wissenschaft', 'lehre', 'immer', 'ja', 'mehr', 'deutschland', 'gibt', 'ermöglicht']
Representative_Docs: ['Immer wieder dieselben Politiksprüche beim Klima Auf Forschung setzen nicht auf Verbote Es ist zum Verzweifeln Ja Mehr Forschung ist toll Aber gleichzeitig müssen wir JETZT umsetzen was die Forschung der letzten Jahrzehnte uns heute ermöglicht Und das ist viel zib', 'Immer wieder dieselben Politiksprüche beim Klima Auf Forschung setzen nicht auf Verbote Es ist zum Verzweifeln Ja Mehr Forschung ist toll Aber gleichzeitig müssen wir JETZT umsetzen was die Forschung der letzten Jahrzehnte uns heute ermöglicht Und das ist viel zib', 'Immer wieder dieselben Politiksprüche beim Klima Auf Forschung setzen nicht auf Verbote Es ist zum Verzweifeln Ja Mehr Forschung ist toll Aber gleichzeitig müssen wir JETZT umsetzen was die Forschung der letzten Jahrzehnte uns heute ermöglicht Und das ist viel zib']

Thema 2:
Representation: ['forschu

In [57]:
model.visualize_topics()

In [58]:
model.visualize_barchart(top_n_topics = 10, n_words = 5)

In [59]:
model.visualize_heatmap()

#### Themen nach Tweet-Kategorie

In [60]:
topics_per_class = model.topics_per_class(docs,
    classes=df.tweet_class)

3it [00:00, 16.11it/s]


In [65]:
topics_per_class

,Topic,Words,Frequency,Class,Name
0,-1,"forschung, the, heute, innovation, gute",134,zitat_tweet,-1_forschung_heute_gibt_mehr
1,0,"forschung, problem, wissenschaft, ichbinhanna, deutschland",73,zitat_tweet,0_forschung_ichbinhanna_wissenschaft_leh...
2,1,"the, fdm, forschungsdatenmanagement, forschung, research",37,zitat_tweet,1_forschung_digitale_the_lehre
3,2,"mecfs, versorgung, longcovid, mehr, mecfsimbundestag",30,zitat_tweet,2_mecfs_versorgung_mecfsimbundestag_fors...
4,3,"berlin, job, freuen, for, gratulieren",18,zitat_tweet,3_berlin_freuen_bewerbungen_exzellente
5,4,"longcovid, covid, heißt, viren, long",13,zitat_tweet,4_heißt_longcovid_covid_viren
6,5,"kernfusion, alternative, akutheit, energiekrise, genakwforschung",30,zitat_tweet,5_kernfusion_wasserstoff_alternative_ene...
7,6,"klima, geht, engagiert, demonstrieren, fleissig",17,zitat_tweet,6_geht_klima_engagiert_ehren
8,7,"frauen, genderstudies, mädchen, mint, wissenschaftlerinnen",11,zitat_tweet,7_frauen_genderstudies_mädchen_wissensch...
9,8,"film, käthe, etymologika, theologie, to",14,zitat_tweet,8_käthe_ukraine_film_leichterstaatspreis


In [62]:
model.visualize_topics_per_class(topics_per_class,
    top_n_topics=10, normalize_frequency = True)

In [63]:
#fig = model.visualize_topics_per_class(topics_per_class)
#fig.write_html("path/to/file.html")